In [1]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 3
STATS_TYPES = ['Count','Percent','nshft', 'pshft','growth', 'pshftna','growthna','Labels']


In [2]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [3]:
STATE_FILES = [
    "tl_2010_01_bg10.json",
    "tl_2010_02_bg10.json",
    "tl_2010_04_bg10.json",
    "tl_2010_05_bg10.json",
    "tl_2010_06_bg10.json",
    "tl_2010_08_bg10.json",
    "tl_2010_09_bg10.json",
    "tl_2010_10_bg10.json",
    "tl_2010_11_bg10.json",
    "tl_2010_12_bg10.json",
    "tl_2010_13_bg10.json",
    "tl_2010_15_bg10.json",
    "tl_2010_16_bg10.json",
    "tl_2010_17_bg10.json",
    "tl_2010_18_bg10.json",
    "tl_2010_19_bg10.json",
    "tl_2010_20_bg10.json",
    "tl_2010_21_bg10.json",
    "tl_2010_22_bg10.json",
    "tl_2010_23_bg10.json",
    "tl_2010_24_bg10.json",
    "tl_2010_25_bg10.json",
    "tl_2010_26_bg10.json",
    "tl_2010_27_bg10.json",
    "tl_2010_28_bg10.json",
    "tl_2010_29_bg10.json",
    "tl_2010_30_bg10.json",
    "tl_2010_31_bg10.json",
    "tl_2010_32_bg10.json",
    "tl_2010_33_bg10.json",
    "tl_2010_34_bg10.json",
    "tl_2010_35_bg10.json",
    "tl_2010_36_bg10.json",
    "tl_2010_37_bg10.json",
    "tl_2010_38_bg10.json",
    "tl_2010_39_bg10.json",
    "tl_2010_40_bg10.json",
    "tl_2010_41_bg10.json",
    "tl_2010_42_bg10.json",
    "tl_2010_44_bg10.json",
    "tl_2010_45_bg10.json",
    "tl_2010_46_bg10.json",
    "tl_2010_47_bg10.json",
    "tl_2010_48_bg10.json",
    "tl_2010_49_bg10.json",
    "tl_2010_50_bg10.json",
    "tl_2010_51_bg10.json",
    "tl_2010_53_bg10.json",
    "tl_2010_54_bg10.json",
    "tl_2010_55_bg10.json",
    "tl_2010_56_bg10.json",
]

In [4]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
# def zip_code_geojson_filename(state_fips_code=1):
#     formatted_fips_code = formatted_state_fips_code(state_fips_code)
#     state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
#     state_full_name = fips_codes[formatted_fips_code]["full_name"]
#     return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


# def zip_geojson_files_for_market(market_data):
#     fips_codes = json.loads(market_data['StateFips'])
#     files = []
#     for fips_code in fips_codes:
#         files.append(zip_code_geojson_filename(fips_code))
#     return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

# def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
#     data_type = "zip"
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)
#     zip_codes = []
#     for data_row in data_rows:
#         zip_codes.append(data_row['Zip'].zfill(5))
#     print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
#     return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "bg"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{data_type}_data_v{MAP_VERSION}.json'

def published_map_settings_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data_v{MAP_VERSION}.json'

def published_map_labels_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_labels_v{MAP_VERSION}.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='bg'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        if block_group in features:
            found = True
            feature = features[block_group]
            new_feature = copy.deepcopy(feature)
            properties = new_feature['properties']
            properties.pop("ALAND10",0)
            properties.pop("AWATER10",0)
            properties.pop("BLKGRPCE10",0)
            properties.pop("COUNTYFP10",0)
            properties.pop("FUNCSTAT10",0)
            properties.pop("MTFCC10",0)
            properties.pop("NAMELSAD10",0)
            properties.pop("STATEFP10",0)
            properties.pop("TRACTCE10",0)
            new_feature['properties'] = properties
            for key in list(data_row.keys())[0:]:
                if key not in  ['BG','LAT','LONG'] and stats_type in key: #and 'Count' in key:
                    if data_row[key] and data_row[key] != 'inf':
                        translated_key = map_settings_data[key]['mapbox_segment']
                        value = ast.literal_eval(data_row[key].replace(',',''))
                        value = int(value * 1000000) / 1000000
                        new_feature['properties'][translated_key] =  value
            new_features_to_keep.append(new_feature)
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)


def create_data_json(features):
    data = []
    for feature in features:
        #centroid = {
        #    "GEOID10": feature["properties"]["GEOID10"],
        #    "INTPTLAT10": feature["properties"]["INTPTLAT10"],
        #    "INTPTLON10": feature["properties"]["INTPTLON10"],
        #}
        datum = feature["properties"]
        data.append(datum)
    return data

def write_geojson(geojson_data, market_prefix, data_type='bg', stats_type='Count',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json')
    
def write_data_json(json_data, market_prefix, data_type='bg', stats_type='count',published_data_base_path = "./public"):
    json_string = json.dumps(json_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote centroid json data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json')


def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "bg"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "bg"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_file_prefix,  data_type , published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            row['CenterLat'] = float(row['CenterLat'])
            row['CenterLong'] = float(row['CenterLong'])
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json')
    
    
def create_tileset_data_recipe_file(market_file_prefix, published_data_base_path, stats_type):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    

def create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_labels_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_labels_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_labels_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
        
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [5]:
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
def create_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    nationwide_features = {}
    for file in STATE_FILES:
        print(f'processing: {file}')
        geosjon_path = os.path.join(geojson_files_base_path,file)
        features = read_geo_gson_file(geosjon_path)
        for feature in features:
            key = feature['properties']["GEOID10"]
            nationwide_features[key] = feature
    json_string = json.dumps(nationwide_features, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson data to  {geojson_files_base_path}/nationwide_bg.json')

def read_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    # Opening JSON file
    f = open(geosjon_path)
    # returns JSON object as 
    # a dictionary
    features = json.load(f)
    # Closing file
    f.close()
    return features

def write_nation_bg_centroids_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    features = read_nation_bg_features_json(geojson_files_base_path)
    centroids = []
    for key in features.keys:
        feature = features[key]
        centroid = {
            "GEOID10": key,
            "INTPTLAT10": feature["properties"]["INTPTLAT10"],
            "INTPTLON10": feature["properties"]["INTPTLON10"],
        }
        centroids.append(centroid)
    json_string = json.dumps(centroids, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg_centroids.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote bg centroids data to  {geojson_files_base_path}/nationwide_bg_centroids.json')
    
        
        
    

In [6]:
def create_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json')



In [7]:
def create_comparison_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json')
    

In [9]:
def create_data_options_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options_v{MAP_VERSION}.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options_v{MAP_VERSION}.json')


In [10]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    nationwide_bg_features = read_nation_bg_features_json(geojson_files_base_path)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows:
        #         if market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        #             #if True:
        #             #print(market_data['MarketName'])
        #if market_data['MarketName'] in ['Ohio','ClevelandOH']:
        #if True: # market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        if market_data['MarketName'] in ['HomosassaSprngFL']:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            market_prefix = market_file_prefix
            mapbox_market_prefix = market_data['MapboxMarketName'].lower()

            for stats_type in STATS_TYPES:
                create_tileset_data_recipe_file(market_prefix, published_data_base_path,stats_type)
                create_tileset_labels_recipe_file(market_prefix, published_data_base_path)
            # Block Group Processing
            data_type = 'bg'
            map_settings_data = read_map_settings(market_data['MarketName'], base_path=source_data_base_path, data_type = "bg")
            map_settings = write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            for stats_type in STATS_TYPES:
                features_to_use = create_bg_features(nationwide_bg_features, map_settings, block_group_rows, stats_type)
                new_geojson = create_geojson(features_to_use)
                write_geojson(new_geojson, market_prefix, data_type, stats_type,published_data_base_path=published_data_base_path)
                if stats_type.lower() != 'labels':
                    new_data_json = create_data_json(features_to_use)
                    write_data_json(new_data_json, market_prefix.lower(),data_type, stats_type.lower(),published_data_base_path)
            create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
# published_data_base_path = "./public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010

In [13]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "./public"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
# Census year for geojson
census_year = 2010


# Process the data
# process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
create_chart_json(published_data_base_path)
create_comparison_chart_json(published_data_base_path)
create_data_options_json(published_data_base_path)

AbileneTX 2012
0.7166328373475683 0.16885358564145184 0.11451357701097988
AbileneTX 2012
0.7860721835120958 0.1372426211539967 0.07668519533390745
AbileneTX 2012
0.6647655828992276 0.1998832405245195 0.13535117657625292
AbileneTX 2012
0.6746865469332429 0.17675364283293798 0.14855981023381903
AbileneTX 2012
0.7392060189803072 0.13907492376528258 0.12171905725441022
AbileneTX 2012
0.692973365115556 0.19831070577261892 0.10871592911182507
AbileneTX 2012
0.675629430457101 0.22708384258127598 0.09728672696162308
AbileneTX 2012
0.8392140866136379 0.08627370997348562 0.07451220341287647
AbileneTX 2012
0.6778640776699029 0.18135922330097087 0.1407766990291262
AbileneTX 2012
0.655952380952381 0.17291666666666666 0.17113095238095238
AbileneTX 2012
0.7345860967821174 0.1885286170474085 0.07688528617047409
AbileneTX 2012
0.6422963058308663 0.2146994465182134 0.14300424765092032
AbileneTX 2012
0.7091821671095364 0.18423383525243578 0.10658399763802776
AbileneTX 2012
0.6515769944341373 0.2604823747

0.7058924387037778 0.29410756129622223 0.0
AltoonaPA 2012
0.6928965985076073 0.3071034014923927 0.0
AltoonaPA 2012
0.7045084577818671 0.2954915422181329 0.0
AltoonaPA 2012
0.6982591876208898 0.30174081237911027 0.0
AltoonaPA 2012
0.681716395327725 0.318283604672275 0.0
AltoonaPA 2012
0.7650738488271068 0.23492615117289314 0.0
AltoonaPA 2012
0.7215095939785858 0.27849040602141417 0.0
AltoonaPA 2012
0.6835079977937121 0.3164920022062879 0.0
AltoonaPA 2012
0.6908077994428969 0.30919220055710306 0.0
AltoonaPA 2012
0.6616868969810147 0.3383131030189854 0.0
AltoonaPA 2012
0.6756001524196621 0.32439984758033785 0.0
AltoonaPA 2012
0.7612410071942446 0.23875899280575538 0.0
AltoonaPA 2012
0.7893772893772893 0.21062271062271062 0.0
AltoonaPA 2012
0.7527027027027027 0.2472972972972973 0.0
AltoonaPA 2012
0.7793764988009593 0.22062350119904076 0.0
AltoonaPA 2017
0.6965826986916618 0.3034173013083382 0.0
AltoonaPA 2017
0.70523311132255 0.29476688867745005 0.0
AltoonaPA 2017
0.693208770995692 0.30679

0.7395898963285411 0.20141474311243485 0.058995360559024
AppletonWI 2017
0.6940897858319605 0.23460667215815487 0.07130354200988467
AppletonWI 2017
0.6832886984467059 0.22983395822174613 0.08687734333154794
AppletonWI 2017
0.7179363957597173 0.20067844522968198 0.08138515901060071
AppletonWI 2017
0.7019152640742891 0.22498549042367963 0.07309924550203134
AppletonWI 2017
0.7062076899252245 0.2471572054333879 0.04663510464138755
AppletonWI 2017
0.7631142098273572 0.1691567065073041 0.06772908366533864
AppletonWI 2017
0.7061521856449001 0.21055945313905378 0.08328836121604605
AppletonWI 2017
0.684085510688836 0.22278646899827995 0.09312802031288393
AppletonWI 2017
0.7231927086929504 0.21038458882828143 0.0664227024787682
AppletonWI 2017
0.6891677675033026 0.23262879788639365 0.07820343461030384
AppletonWI 2017
0.6781062642133554 0.24477982220384537 0.07711391358279926
AppletonWI 2017
0.7341123166805771 0.23226421843328962 0.033623464886133304
AppletonWI 2017
0.6893566322478157 0.258220810

0.22340533844665797 0.555157726833076 0.22143693472026607
AustinTX 2017
0.2615111505603018 0.435666999519213 0.3028218499204852
AustinTX 2020
0.28455879099253106 0.4739948809554308 0.2414463280520382
AustinTX 2020
0.3438990043728537 0.4606270227536626 0.1954739728734837
AustinTX 2020
0.23216041455158035 0.5147145954567145 0.25312498999170513
AustinTX 2020
0.230529902711245 0.422468834439796 0.34700126284895905
AustinTX 2020
0.34086612755614903 0.40447916822948704 0.2546547042143639
AustinTX 2020
0.2683597559380803 0.48073323195223383 0.25090701210968586
AustinTX 2020
0.2516571393352953 0.5255512241722007 0.22279163649250397
AustinTX 2020
0.40876294703928084 0.38871213601719756 0.2025249169435216
AustinTX 2020
0.3104298438105073 0.4338195430489222 0.25575061314057057
AustinTX 2020
0.23749938742506166 0.40029730303673816 0.3622033095382002
AustinTX 2020
0.32763631435498203 0.47242843590477196 0.19993524974024604
AustinTX 2020
0.21436482084690553 0.5144733984799131 0.2711617806731813
Aust

0.7377652663490689 0.22650498051104373 0.035729753139887396
BayCityMI 2017
0.7016154085119602 0.25287356321839083 0.04551102826964896
BayCityMI 2017
0.8172791938352104 0.14508002371072912 0.03764078245406046
BayCityMI 2017
0.8055624844100773 0.14928909952606634 0.04514841606385633
BayCityMI 2017
0.746742277992278 0.2101833976833977 0.04307432432432432
BayCityMI 2017
0.7379838366652488 0.2237345810293492 0.038281582305401955
BayCityMI 2017
0.7138994910941476 0.25127226463104324 0.03482824427480916
BayCityMI 2017
0.789804469273743 0.17667597765363127 0.0335195530726257
BayCityMI 2017
0.7006802721088435 0.2599611273080661 0.03935860058309038
BayCityMI 2017
0.6882972823072657 0.25513033832501386 0.056572379367720464
BayCityMI 2017
0.7661498708010336 0.2235142118863049 0.0103359173126615
BayCityMI 2020
0.7592227218500505 0.20044507662659447 0.04033220152335505
BayCityMI 2020
0.764831444133667 0.19954364787281026 0.03562490799352274
BayCityMI 2020
0.7486681107663485 0.20818453252151514 0.043

...

SanAngeloTX-2020-1:All-2:$0-50k
SanAngeloTX-2020-1:All-3:$50-100k
SanAngeloTX-2020-1:All-4:$100k+
SanAngeloTX-2020-2:18-44-2:$0-50k
SanAngeloTX-2020-3:45-64-2:$0-50k
SanAngeloTX-2020-4:65+-2:$0-50k
SanAngeloTX-2020-2:18-44-3:$50-100k
SanAngeloTX-2020-3:45-64-3:$50-100k
SanAngeloTX-2020-4:65+-3:$50-100k
SanAngeloTX-2020-2:18-44-4:$100k+
SanAngeloTX-2020-3:45-64-4:$100k+
SanAngeloTX-2020-4:65+-4:$100k+
SanAngeloTX-2022-1:All-1:All
SanAngeloTX-2022-2:18-44-1:All
SanAngeloTX-2022-3:45-64-1:All
SanAngeloTX-2022-4:65+-1:All
SanAngeloTX-2022-1:All-2:$0-50k
SanAngeloTX-2022-1:All-3:$50-100k
SanAngeloTX-2022-1:All-4:$100k+
SanAngeloTX-2022-2:18-44-2:$0-50k
SanAngeloTX-2022-3:45-64-2:$0-50k
SanAngeloTX-2022-4:65+-2:$0-50k
SanAngeloTX-2022-2:18-44-3:$50-100k
SanAngeloTX-2022-3:45-64-3:$50-100k
SanAngeloTX-2022-4:65+-3:$50-100k
SanAngeloTX-2022-2:18-44-4:$100k+
SanAngeloTX-2022-3:45-64-4:$100k+
SanAngeloTX-2022-4:65+-4:$100k+
SanAntonioTX-2012-1:All-1:All
SanAntonioTX-2012-2:18-44-1:All
SanAntonio

SumterSC-2022-1:All-1:All
SumterSC-2022-2:18-44-1:All
SumterSC-2022-3:45-64-1:All
SumterSC-2022-4:65+-1:All
SumterSC-2022-1:All-2:$0-50k
SumterSC-2022-1:All-3:$50-100k
SumterSC-2022-1:All-4:$100k+
SumterSC-2022-2:18-44-2:$0-50k
SumterSC-2022-3:45-64-2:$0-50k
SumterSC-2022-4:65+-2:$0-50k
SumterSC-2022-2:18-44-3:$50-100k
SumterSC-2022-3:45-64-3:$50-100k
SumterSC-2022-4:65+-3:$50-100k
SumterSC-2022-2:18-44-4:$100k+
SumterSC-2022-3:45-64-4:$100k+
SumterSC-2022-4:65+-4:$100k+
SyracuseNY-2012-1:All-1:All
SyracuseNY-2012-2:18-44-1:All
SyracuseNY-2012-3:45-64-1:All
SyracuseNY-2012-4:65+-1:All
SyracuseNY-2012-1:All-2:$0-50k
SyracuseNY-2012-1:All-3:$50-100k
SyracuseNY-2012-1:All-4:$100k+
SyracuseNY-2012-2:18-44-2:$0-50k
SyracuseNY-2012-3:45-64-2:$0-50k
SyracuseNY-2012-4:65+-2:$0-50k
SyracuseNY-2012-2:18-44-3:$50-100k
SyracuseNY-2012-3:45-64-3:$50-100k
SyracuseNY-2012-4:65+-3:$50-100k
SyracuseNY-2012-2:18-44-4:$100k+
SyracuseNY-2012-3:45-64-4:$100k+
SyracuseNY-2012-4:65+-4:$100k+
SyracuseNY-2017-

WichitaFallsTX-2012-1:All-1:All
WichitaFallsTX-2012-2:18-44-1:All
WichitaFallsTX-2012-3:45-64-1:All
WichitaFallsTX-2012-4:65+-1:All
WichitaFallsTX-2012-1:All-2:$0-50k
WichitaFallsTX-2012-1:All-3:$50-100k
WichitaFallsTX-2012-1:All-4:$100k+
WichitaFallsTX-2012-2:18-44-2:$0-50k
WichitaFallsTX-2012-3:45-64-2:$0-50k
WichitaFallsTX-2012-4:65+-2:$0-50k
WichitaFallsTX-2012-2:18-44-3:$50-100k
WichitaFallsTX-2012-3:45-64-3:$50-100k
WichitaFallsTX-2012-4:65+-3:$50-100k
WichitaFallsTX-2012-2:18-44-4:$100k+
WichitaFallsTX-2012-3:45-64-4:$100k+
WichitaFallsTX-2012-4:65+-4:$100k+
WichitaFallsTX-2017-1:All-1:All
WichitaFallsTX-2017-2:18-44-1:All
WichitaFallsTX-2017-3:45-64-1:All
WichitaFallsTX-2017-4:65+-1:All
WichitaFallsTX-2017-1:All-2:$0-50k
WichitaFallsTX-2017-1:All-3:$50-100k
WichitaFallsTX-2017-1:All-4:$100k+
WichitaFallsTX-2017-2:18-44-2:$0-50k
WichitaFallsTX-2017-3:45-64-2:$0-50k
WichitaFallsTX-2017-4:65+-2:$0-50k
WichitaFallsTX-2017-2:18-44-3:$50-100k
WichitaFallsTX-2017-3:45-64-3:$50-100k
Wi

Wrote market summary charts data to  /Users/alanmccann/Dropbox/bain/mtsds-data/market_comparison_charts_v3.json
Wrote data options data to  /Users/alanmccann/Dropbox/bain/mtsds-data/data_options_v3.json


/var/folders/dh/fn0cs6ln6sq0722wd5lcgdcm0000gn/T/ipykernel_79541/1269797648.py:28: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')


In [16]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src_v{MAP_VERSION} {market_file_prefix}_bg_data_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION} --recipe {market_file_prefix}-bg-recipe_v{MAP_VERSION}.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION}')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels_v{MAP_VERSION}"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [17]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}

import subprocess
# tilesets job stobieb.dallasfortworth_bg_data cl9kgdofo000209l52zyy3mdi
process = subprocess.Popen(['tilesets', 'job', 'stobieb.dallasfortworth_bg_data', 'cl9kgdofo000209l52zyy3mdi'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)
stdout, stderr = process.communicate()
output = stdout.decode('utf-8').split('\n')
error = stderr.decode('utf-8')
result = json.loads(output[0])



In [18]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_v{MAP_VERSION}.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [19]:
def update_recipe(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', 
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [20]:
import time
def publish_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())


In [31]:
import csv
import datetime
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
# STATS_TYPES = ['count','percent','nshft','labels','pshft','pshftna','growthna']
STATS_TYPES = ['growth']
# MARKETS = ["ColoradoSpringCO","DallasFortWrthTX","EastStroudsbrgPA","HiltonHeadIlndSC","HomosassaSprngFL","DstrctOfColumbia"]
# MARKETS = ["ClevelandOH"]

# STATS_TYPES = ['pshftna','growthna']
# STATS_TYPES = ['nshft']
# Ohio market_rows[416:417]
# Cleveland market_rows[73:74]
count = 77
for market_data in market_rows[77:]: #[416:417]:
    market_file_prefix = market_data['MapboxMarketName'].lower()
    if True: #market_data['MarketName'] in MARKETS: #,'DallasFortWorthTX']:
        count += 1
        print('count',count)
        print('creating tilesets for:', market_file_prefix)
        print(datetime.datetime.now())
        for stats_type in STATS_TYPES:
            upload_source(market_file_prefix, stats_type, published_data_base_path)
            upload_tileset(market_file_prefix, stats_type, published_data_base_path)
            update_recipe(market_file_prefix, stats_type, published_data_base_path)
            publish_tileset(market_file_prefix, stats_type, published_data_base_path)
        
        

count 78
creating tilesets for: columbiamo
2023-07-21 06:22:52.392402
tilesets upload-source stobieb columbiamo_growth_src_v3 columbiamo_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/columbiamo_growth_src_v3", "files": 1, "source_size": 1011943, "file_size": 1011943}', '']
tilesets create stobieb.columbiamo_growth_v3 --recipe columbiamo_growth_recipe_v3.json --name columbiamo_growth_v3
['{"message": "stobieb.columbiamo_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.columbiamo_growth_v3 columbiamo_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.columbiamo_growth_v3
2023-07-21 06:22:58.958321
['{"message": "Processing stobieb.columbiamo_growth_v3", "jobId": "clkcfnky8002108k8bj3k8yk2"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.columbiamo_growth_v3 or run tilesets job stobieb.columbiamo_growth_v3 clkcfnky8002108k8bj3k8yk2 to view the status of your tileset.

{'message': 'Pr

['upload progress', '{"id": "mapbox://tileset-source/stobieb/corvallisor_growth_src_v3", "files": 1, "source_size": 506182, "file_size": 506182}', '']
tilesets create stobieb.corvallisor_growth_v3 --recipe corvallisor_growth_recipe_v3.json --name corvallisor_growth_v3
['{"message": "stobieb.corvallisor_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.corvallisor_growth_v3 corvallisor_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.corvallisor_growth_v3
2023-07-21 06:40:40.298613
['{"message": "Processing stobieb.corvallisor_growth_v3", "jobId": "clkcgace6000b08l2akdeffhv"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.corvallisor_growth_v3 or run tilesets job stobieb.corvallisor_growth_v3 clkcgace6000b08l2akdeffhv to view the status of your tileset.

{'message': 'Processing stobieb.corvallisor_growth_v3', 'jobId': 'clkcgace6000b08l2akdeffhv'}
processing: sleeping
processing: sleeping
processing: sleeping
processing:

['{"message": "stobieb.daphneal_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.daphneal_growth_v3 daphneal_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.daphneal_growth_v3
2023-07-21 06:58:06.317419
['{"message": "Processing stobieb.daphneal_growth_v3", "jobId": "clkcgwr9d000308l7dffc926e"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.daphneal_growth_v3 or run tilesets job stobieb.daphneal_growth_v3 clkcgwr9d000308l7dffc926e to view the status of your tileset.

{'message': 'Processing stobieb.daphneal_growth_v3', 'jobId': 'clkcgwr9d000308l7dffc926e'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 07:00:53.834644
count 91
creating tilesets for: davenportia
2023-07-21 07:00:53.834805
tilesets upload-source stobieb davenportia_growth_

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 07:18:35.941325
count 97
creating tilesets for: desmoinesia
2023-07-21 07:18:35.941447
tilesets upload-source stobieb desmoinesia_growth_src_v3 desmoinesia_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/desmoinesia_growth_src_v3", "files": 1, "source_size": 1668926, "file_size": 1668926}', '']
tilesets create stobieb.desmoinesia_growth_v3 --recipe desmoinesia_growth_recipe_v3.json --name desmoinesia_growth_v3
['{"message": "stobieb.desmoinesia_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.desmoinesia_growth_v3 desmoinesia_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.desmoinesia_growth_v3
2023-07-21 07:18:42.259739
['{"message": "Processing stobieb.desmoinesia_growth_v3", "jobId":

['upload progress', '{"id": "mapbox://tileset-source/stobieb/durhamnc_growth_src_v3", "files": 1, "source_size": 3161462, "file_size": 3161462}', '']
tilesets create stobieb.durhamnc_growth_v3 --recipe durhamnc_growth_recipe_v3.json --name durhamnc_growth_v3
['{"message": "stobieb.durhamnc_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.durhamnc_growth_v3 durhamnc_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.durhamnc_growth_v3
2023-07-21 07:36:59.001859
['{"message": "Processing stobieb.durhamnc_growth_v3", "jobId": "clkciar4e000r08ky4eqn0id8"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.durhamnc_growth_v3 or run tilesets job stobieb.durhamnc_growth_v3 clkciar4e000r08ky4eqn0id8 to view the status of your tileset.

{'message': 'Processing stobieb.durhamnc_growth_v3', 'jobId': 'clkciar4e000r08ky4eqn0id8'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
pro

...

['{"message": "stobieb.michigan_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.michigan_growth_v3 michigan_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.michigan_growth_v3
2023-07-21 23:37:25.091242
['{"message": "Processing stobieb.michigan_growth_v3", "jobId": "clkdglvwc003u08jl4b9xfp68"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.michigan_growth_v3 or run tilesets job stobieb.michigan_growth_v3 clkdglvwc003u08jl4b9xfp68 to view the status of your tileset.

{'message': 'Processing stobieb.michigan_growth_v3', 'jobId': 'clkdglvwc003u08jl4b9xfp68'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 23:40:54.833303
count 405
creating tilesets for: minnesota
2023-07-21 23:40:54.833420
tilesets upload-source stobieb minnesota_growth_src

['upload progress', '{"id": "mapbox://tileset-source/stobieb/newhampshire_growth_src_v3", "files": 1, "source_size": 6457931, "file_size": 6457931}', '']
tilesets create stobieb.newhampshire_growth_v3 --recipe newhampshire_growth_recipe_v3.json --name newhampshire_growth_v3
['{"message": "stobieb.newhampshire_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.newhampshire_growth_v3 newhampshire_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.newhampshire_growth_v3
2023-07-21 23:53:03.175322
['{"message": "Processing stobieb.newhampshire_growth_v3", "jobId": "clkdh5z6h005o08jl9dq42ue0"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.newhampshire_growth_v3 or run tilesets job stobieb.newhampshire_growth_v3 clkdh5z6h005o08jl9dq42ue0 to view the status of your tileset.

{'message': 'Processing stobieb.newhampshire_growth_v3', 'jobId': 'clkdh5z6h005o08jl9dq42ue0'}
queued: sleeping
finished
2023-07-21 23:53:22.025515
count 4

['upload progress', '{"id": "mapbox://tileset-source/stobieb/oklahoma_growth_src_v3", "files": 1, "source_size": 29285976, "file_size": 29285976}', '']
tilesets create stobieb.oklahoma_growth_v3 --recipe oklahoma_growth_recipe_v3.json --name oklahoma_growth_v3
['{"message": "stobieb.oklahoma_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.oklahoma_growth_v3 oklahoma_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.oklahoma_growth_v3
2023-07-22 00:10:58.422925
['{"message": "Processing stobieb.oklahoma_growth_v3", "jobId": "clkdht11y002k08jz38hx9ddo"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.oklahoma_growth_v3 or run tilesets job stobieb.oklahoma_growth_v3 clkdht11y002k08jz38hx9ddo to view the status of your tileset.

{'message': 'Processing stobieb.oklahoma_growth_v3', 'jobId': 'clkdht11y002k08jz38hx9ddo'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
p

['{"message": "Processing stobieb.tennessee_growth_v3", "jobId": "clkdj3g0k000208jm9otjdpzp"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.tennessee_growth_v3 or run tilesets job stobieb.tennessee_growth_v3 clkdj3g0k000208jm9otjdpzp to view the status of your tileset.

{'message': 'Processing stobieb.tennessee_growth_v3', 'jobId': 'clkdj3g0k000208jm9otjdpzp'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-22 00:50:05.286513
count 425
creating tilesets for: texas
2023-07-22 00:50:05.286639
tilesets upload-source stobieb texas_growth_src_v3 texas_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/texas_growth_src_v3", "files": 1, "source_size": 126056478, "file_size": 126056478}', '']
tilesets create stobieb.texas_growth_v3 --recipe texas_gro

processing: sleeping
processing: sleeping
finished
2023-07-22 01:20:10.741744
count 431
creating tilesets for: wisconsin
2023-07-22 01:20:10.742014
tilesets upload-source stobieb wisconsin_growth_src_v3 wisconsin_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/wisconsin_growth_src_v3", "files": 1, "source_size": 37520014, "file_size": 37520014}', '']
tilesets create stobieb.wisconsin_growth_v3 --recipe wisconsin_growth_recipe_v3.json --name wisconsin_growth_v3
['{"message": "stobieb.wisconsin_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.wisconsin_growth_v3 wisconsin_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.wisconsin_growth_v3
2023-07-22 01:20:29.406485
['{"message": "Processing stobieb.wisconsin_growth_v3", "jobId": "clkdkafb4005t08l2bsor5jeu"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.wisconsin_growth_v3 or run tilesets job stobieb.wisconsin_growth_v3 clkdkafb400

In [16]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)

In [61]:
for row in market_rows:
    print(row["MarketName"],len(row["MarketName"]))

AbileneTX 9
AkronOH 7
AlbanyGA 8
AlbanyOR 8
AlbanyNY 8
AlbuquerqueNM 13
AlexandriaLA 12
AllentownPA 11
AltoonaPA 9
AmarilloTX 10
AmesIA 6
AnchorageAK 11
AnnArborMI 10
AnnistonAL 10
AppletonWI 10
AshevilleNC 11
AthensGA 8
AtlantaGA 9
AtlanticCityNJ 14
AuburnAL 8
AugustaGA 9
AustinTX 8
BakersfieldCA 13
BaltimoreMD 11
BangorME 8
BarnstableTownMA 16
BatonRougeLA 12
BattleCreekMI 13
BayCityMI 9
BeaumontTX 10
BeckleyWV 9
BellinghamWA 12
BendOR 6
BillingsMT 10
BinghamtonNY 12
BirminghamAL 12
BismarckND 10
BlacksburgVA 12
BloomingtonIL 13
BloomingtonIN 13
BloomsburgPA 12
BoiseCityID 11
BostonMA 8
BoulderCO 9
BowlingGreenKY 14
BremertonWA 11
BridgeportCT 12
BrownsvilleTX 13
BrunswickGA 11
BuffaloNY 9
BurlingtonNC 12
BurlingtonVT 12
CaliforniaMD 12
CantonOH 8
CapeCoralFL 11
CapeGirardeauMO 15
CarbondaleIL 12
CarsonCityNV 12
CasperWY 8
CedarRapidsIA 13
ChambersburgPA 14
ChampaignIL 11
CharlestonWV 12
CharlestonSC 12
CharlotteNC 11
CharlottesvilleVA 17
ChattanoogaTN 13
CheyenneWY 10
ChicagoIL 9
Ch

In [46]:
market_rows[155]['MarketName']

'HomosassaSprngFL'